## En este ejercicio vamos generar reglas de asociación, con el algoritmo "apriori" ##

<br>
<div class="alert alert-block alert-info">


El  <a href="https://es.wikipedia.org/wiki/Algoritmo_apriori" class="alert-link">algoritmo a priori</a> se usa en el aprendizaje no supervisado, para establecer relaciones entre los objetos. <br>

Tambien se llama Basket Case Analisys.
<br>
Se usa como un punto de partida para encontrar patrones ocultos entre las features.
</div>


En este ejercicio, vamos a usar un dataset que hice para el sistema de recomendación de repositorios de github. <br>Cada fila es un usuario, y cada columna es un boolean, si el usuario tiene esa habilidad en sus repos, o en sus favoritos.

### Fases del ejercicio.###

 - Descargar el dataset de github, y cargarlo con pickle.
 - Preprocesado : Eliminar del dataset columnas redundantes.
 - Hallar items frecuentes.
 - Hallar reglas de asociacion

### Descargar el dataset ###

In [2]:
# Descargamos el dataset
! [ ! -f datasets/Users_Tag_Matrix.data ] && \\
wget https://raw.githubusercontent.com/jaimevalero/github-recommendation-engine/master/Users_Tag_Matrix.data.gz -O datasets/Users_Tag_Matrix.data.gz 
#  y descomprimimos    
! [ ! -f datasets/Users_Tag_Matrix.data ] && gunzip ./datasets/Users_Tag_Matrix.data.gz
    


--2018-05-16 18:58:11--  https://raw.githubusercontent.com/jaimevalero/github-recommendation-engine/master/Users_Tag_Matrix.data.gz
Resolviendo raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.36.133
Conectando con raw.githubusercontent.com (raw.githubusercontent.com)[151.101.36.133]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 5165370 (4,9M) [application/octet-stream]
Grabando a: “datasets/Users_Tag_Matrix.data.gz”

100%[======================================>] 5.165.370   4,01MB/s   en 1,2s   

2018-05-16 18:58:13 (4,01 MB/s) - “datasets/Users_Tag_Matrix.data.gz” guardado [5165370/5165370]



### Cargar el dataset con pickle ###

In [3]:
import pandas as pd
import pickle
    
df = pickle.load( open( "datasets/Users_Tag_Matrix.data", "rb" ) )


    


### Preprocesado : Eliminar del dataset columnas redundantes. ###


In [4]:
# Borramos las columnas no deseadas
COLUMNS_TO_DELETE = ["c","resources","examples","components","iphone","awesome-lists", 
                     "package-manager", "material-design", "systems", "slides", "language",
                    "programming", "web-app", "docker-image" , "asynchronous" , "scalable" ,
                    "web-framework","angular" ,"library", "web-development", "dockerfile",
                    "videos" , "streaming" , "deployment" , "simple", "search"]
try:
  for column in COLUMNS_TO_DELETE :   del df[column]
except: pass

# Pasamos a boolean
df  = df.astype(bool)

for column in df.columns : print (column)

assembly
batchfile
c#
c++
clojure
coffeescript
css
elixir
emacs lisp
go
haskell
html
java
javascript
jupyter notebook
kotlin
lua
matlab
objective-c
objective-c++
ocaml
perl
php
powershell
purebasic
python
rascal
ruby
rust
scala
shell
swift
tex
typescript
vim script
vue
1-wire
2d
3d
3d-engine
3d-game-engine
accessibility
accordion
acme
acme-client
activejob
activerecord
activity
activity-stream
actor-model
adc
addons
admin
admin-dashboard
admin-template
admin-theme
admin-ui
ado-net
adobe
after-effects
ag
agc
agent
airbnb
airtable
akka
alarm
alerting
algorithm
algorithm-challenges
algorithm-competitions
alignment
amd
analytics
android
android-application
android-architecture
android-cleanarchitecture
android-development
android-interview-questions
android-library
android-testing
android-ui
angular-2
angular-components
angular2
angular4
angularclass
angularjs
angularjs-interview-questions
animation
animation-library
anonymity
ansible
antd
anticensorship
anyconnect
aot
aot-compilation
apac

In [5]:
# Imprime el dataset listo para usarse
df.head()


assembly  batchfile     c#    c++  clojure  coffeescript  \
007lva.json         False      False  False  False    False         False   
06wj.json           False      False   True  False    False         False   
0bserver07.json     False      False  False  False    False         False   
0rca.json           False      False  False  False     True         False   
0x00A.json          False      False  False   True    False         False   

                   css  elixir  emacs lisp     go        ...          \
007lva.json      False   False       False   True        ...           
06wj.json        False   False       False  False        ...           
0bserver07.json  False   False       False  False        ...           
0rca.json        False   False       False  False        ...           
0x00A.json        True   False       False  False        ...           

                 yeoman-generator    yii   yii2  youtube  zephir  \
007lva.json                 False  False  False    False   False   
06wj.json                   False  False  False    False   False   
0bserver07.json             False  False  False    False   False   
0rca.json                   False  False  False    False   False   
0x00A.json                  False  False  False    False   False   

                 zero-configuration  zeromq  zookeeper    zsh  \
007lva.json                   False   False      False  False   
06wj.json                     False   False      False  False   
0bserver07.json               False   False      False  False   
0rca.json                     False   False      False  False   
0x00A.json                    False   False      False  False   

                 zsh-configuration  
007lva.json                  False  
06wj.json                    False  
0bserver07.json              False  
0rca.json                    False  
0x00A.json                   False  

[5 rows x 1384 columns]

### Hallar items frecuentes. ###




In [6]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from IPython.display import display, HTML

### Generamos los items mas frecuentes, usando el algoritmo apriori
frequent_itemsets = apriori(df, min_support=0.03, use_colnames=True)

metrics = { "support" : """The support metric is defined for itemsets, not assocication rules, and computes the proportion of transactions that contain the antecedant"""  }


# Imprimimos en forma de tabla HTML
for key, value in metrics.items():
    s = f"""
      <h1> Metric : {key}</h1> 
      <table>
        <tr>
          <th>Metric</th>
          <th>Meaning</th>
        </tr>
        <tr>
          <td>{key}</td>
          <td>{value}</td>
        </tr>
      </table>
      <br>
      The rules sort by {key} are :"""
    display(HTML(s))
    display(HTML(frequent_itemsets.sort_values(key,ascending=False).head(20).to_html()))



### Hallar reglas de asociación. ###


In [7]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from IPython.display import display, HTML

### Generamos las reglas de asociacion
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1).sort_values("confidence",ascending=False)


### Ordenamos las reglas de asociación, por cada métrica. ###



In [8]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from IPython.display import display, HTML

from IPython.display import display, HTML



metrics = {
           "confidence"  : """The confidence of a rule A->C is the probability of seeing the consequent in a transaction given that it also contains the antecedent. Note that the metric is not symmetric or directed; for instance, the confidence for A->C is different than the confidence for C->A. """ ,
           "leverage"    : """Leverage computes the difference between the observed frequency of A and C appearing together and the frequency that would be expected if A and C were independent. An leverage value of 0 indicates independence. """ ,
           "lift"        : """The lift metric is commonly used to measure how much more often the antecedent and consequent of a rule A->C occur together than we would expect if they were statistically independent. """,
           "conviction"  : """A high conviction value means that the consequent is highly depending on the antecedent. For instance, in the case of a perfect confidence score, the denominator becomes 0 (due to 1 - 1) """ 
         }

# Imprimimos en forma de tabla HTML
for key, value in metrics.items():
    s = f"""
      <h1> Metric : {key}</h1> 
      <table>
        <tr>
          <th>Metric</th>
          <th>Meaning</th>
        </tr>
        <tr>
          <td>{key}</td>
          <td>{value}</td>
        </tr>
      </table>
      <br>
      The rules sort by {key} are :"""
    display(HTML(s))
    display(HTML(rules.sort_values(key,ascending=False).head(20).to_html()))
